# Entornos 

Los códigos de Python se ejecutan en un entorno virtual, que define la versión del intérprete de Python y los paquetes instalados disponibles. Los entornos suelen gestionarse con Conda o Pip. Para mejorar la portabilidad, se suele crear entornos en contenedores **Docker** alojados en objetivos de cómputo (equipos de desarrollo, máquinas virtuales o clusters en la nube).

AzureML gestiona la creación de los entornos y la instalación de paquetes, generalmente mediante contenedores Docker. Puedes especificar los paquetes necesarios y que AzureML cree un entorno para el experimento.

Dentro de las soluciones empresariales, es importante conocer los entornos de ejecución del código. Los entornos están encapsulados en la clase `environment`, que permite crearlos y especificar la configuración de ejecución.

Opciones de gestión de entornos:

- Azure ML: Crea y registra el entorno automáticamente.
- Gestión manual: Crea y registra entornos propios.

##### Creación de un entorno a partir de un archivo de especificación
Podemos utilizar un archivo de especificación Conda o pip para definir los paquetes necesarios en un entorno de Python y utilizarlo para crear un objeto `Environment`.

In [ ]:
# Ejemplo de archivo de especificación de entorno que se podria llamar conda.yml

name: py_env
dependencies:
  - numpy
  - pandas
  - scikit-learn
  - pip:
    - azureml-defaults

In [ ]:
# Codigo para trabajar con el archivo conda.yml cy sus variables de entorno.

from azureml.core import Environment

env = Environment.from_conda_specification(name='training_environment',
                                           file_path='./conda.yml')

##### Creación de un entorno a partir de un entorno de Conda existente

Si ya tenemos un entorno de Conda existente definido en la estación de trabajo, podemos usarlo para definir nuestro nuevo entorno.

In [ ]:
from azureml.core import Environment

env = Environment.from_existing_conda_environment(name='training_environment',
                                                  conda_environment_name='py_env')

##### Creación de un entorno mediante la especificación de paquetes
Podemos definir un entorno especificando los paquetes Conda y pip que necesitamos en un objeto CondaDependencies

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment('training_environment')
deps = CondaDependencies.create(conda_packages=['scikit-learn','pandas','numpy'],
                                pip_packages=['azureml-defaults'])
env.python.conda_dependencies = deps

##### Configuración de contenedores de entorno

Normalmente, los entornos para el script del experimento se crean en contenedores. 

En el código siguiente se configura un experimento basado en scripts para hospedar el entorno de entorno creado anteriormente en un contenedor (este es el valor predeterminado a menos que use `DockerConfiguration` con un  atributo `use_docker=False`, en cuyo caso el entorno se crea directamente en el destino de proceso)

In [ ]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration

docker_config = DockerConfiguration(use_docker=True)

script_config = ScriptRunConfig(source_directory='my_folder',
                                script='my_script.py',
                                environment=env,
                                docker_runtime_config=docker_config)

Azure ML usa una biblioteca de imágenes base para contenedores, eligiendo la base adecuada para el destino de proceso que se especifique (por ejemplo, incluida la compatibilidad de Cuda con el proceso basado en GPU). Si creamos imágenes de contenedor personalizadas y las hemos registrado en un registro de contenedor, podemos invalidar las imágenes base creadas y usar las nuestras propias modificando los atributos de la propiedad docker del entorno.

In [ ]:
env.docker.base_image='my-base-image'
env.docker.base_image_registry='myregistry.azurecr.io/myimage'

Como alternativa, podemos crear una imagen a petición en función de la imagen base y la configuración adicional de un `dockerfile`

In [ ]:
env.docker.base_image = None
env.docker.base_dockerfile = './Dockerfile'

Por defecto, Azure Machine Learning controla las rutas de acceso de Python y las dependencias de paquetes.
Si la imagen ya incluye una instalación de Python con las dependencias que necesita, podemos invalidar este comportamiento estableciendo `python.user_managed_dependencies=True` y estableciendo una ruta de acceso explícita de Python para la instalación.

In [ ]:
env.python.user_managed_dependencies=True
env.python.interpreter_path = '/opt/miniconda/bin/python'

#### Registro y reutilización de entornos
Con el entorno creado, podemos registrarlo en el área de trabajo y reutilizarlo para futuros experimentos que tengan las mismas dependencias de Python.

##### Registro de un entorno
usamos el metodo `register` del objeto `Environment` para registrar un entorno

In [ ]:
env.register(workspace=ws)

Podemos ver los entornos registrados de la siguiente manera

In [ ]:
from azureml.core import Environment

env_names = Environment.list(workspace=ws)
for env_name in env_names:
    print('Name:',env_name)

##### Recuperación y uso de un entorno
Podemos recuperar un entorno registrado mediante el método `get` de la  clase `Environment` y, a continuación, asignarlo a `ScriptRunConfig`.

In [ ]:
#  En este ejemplo se recupera el entorno registrado training_environment y se asigna a una configuración de ejecución de script:

from azureml.core import Environment, ScriptRunConfig

training_env = Environment.get(workspace=ws, name='training_environment')

script_config = ScriptRunConfig(source_directory='my_folder',
                                script='my_script.py',
                                environment=training_env)